# Protein Products Advertising Campaign (Data Science Case Study)

## 1 Introduction

## 1.1 Description & Discussion of the Background  

### 1.1.1 General Discussion

Based on BBC <a href='http://www.bbc.co.uk/newsbeat/article/38555693/protein-supplement-advert-claims-wrong-and-immoral'>'Protein Supplement Advert Claims Wrong and Immoral'</a> article, The British Dietetic Association (BDA) believes marketing for some products is both "wrong and immoral". In which It means that thousands of people are using protein powders as a <strong>"Substitute not a supplement"</strong>. But the body which represents the sports nutrition industry says extra protein allows people to train harder.  
The ESSNA (European Specialist Sports Nutrition Alliance) also argues that synthetic products enable faster recovery.  
There's been a 20% year-on-year increase in sales of protein products over the last five years, according to Euromonitor figures obtained by Radio 1 Newsbeat.
That makes them the fastest growing type of sport supplement in the UK.
UK consumers spent £66m on sports nutrition food and drink products in 2015, up by 27% from 2013.
A statistical review by "Protein Supplements Market Size & Growth, Industry Report 2018-2025" described the annual protein supplements revenues by Product:

<img src="https://www.grandviewresearch.com/static/img/research/us-protein-supplements-market.png" />

### 1.1.2 Background

A company called Mass-Protein wants to start an advertising campaign for their new protein product in the city of Toronto and after several meetings the company made the plane of the process that need to be followed in order to get the highest value from there campaign, and it was as follow: -  
<ul >
    <li>Create a map which illustrate the administrative division Toronto city.</li>
    <li>Identifying the Boroughs including neighborhoods in Toronto.</li>
    <li>Identifying the ratio of gyms in each Boroughs.</li>
    <li>Start a Facebook, YouTube advertising campaign that targets people by their location.</li>
<ul>

## 1.2 Problem Statement

The company decided to invest a 100,000$ in this campaign and haired a Data Scientist to answer the following question: -  
<strong><font size="3em">What's the amount of budget will we invest in each area in the city of Toronto for the advertising campaign?</font></strong>

# 2. Implementation / Code

### 2.1 Installing Required Library's  
Please if you haven't run the following code to install packages the remove the comments and run the code.

In [7]:
! pip install geocoder                              #For Installing GeoCoder Library
! conda install -c conda-forge geopy --yes          #For Installing Geopy Library
! conda install -c conda-forge folium=0.5.0 --yes   #For Installing Folium

## 2.2. Importing Librarys

In [9]:
import numpy as np                        #For Vectorized Computation
import os as os                           #For File Proccessing
import folium                             #For Map Rendering
import requests                           #For HTTP Requests
import lxml.html as lh                    #For Html to ? Convertion
import pandas as pd                       #For Creating Dataframs
from lxml import etree                    #For Converting HTML Elements to Raw HTML
import geocoder                           #For Getting the latitude and longitude Coordinates
from geopy.geocoders import Nominatim     #For Converting an Address into latitude and longitude Values
import json                               #For handleing JSON files
from pandas.io.json import json_normalize #For Tranforming JSON File Into a Pandas Dataframe
from sklearn.cluster import KMeans        #For import K-means From Clustering Stage
import matplotlib.cm as cm                #For Matplotlib and associated plotting modules
import matplotlib.colors as colors        #For Matplotlib and associated plotting modules
from math import sin, cos, sqrt, atan2, radians #For Mathematical Operations
print('Importing Librarys Complete')      

Importing Librarys Complete


### 2.3. Loading Data  
### 2.3.1. Loading Spatial Data

#### GIS Data (Second-level Administrative Divisions, Canada, 2015) - Loading Data

In [10]:
#Creating  File Structer
if(os.path.isdir('Data/SpatialData/') == False):
   os.mkdir("Data/SpatialData/")                                            #Create Data Directory

 
url = 'https://geo.nyu.edu/download/file/stanford-pg953dp1700-geojson.json' #Second-Level Administrative Spatial Data
GeoJSON_File = 'Data/SpatialData/Canada.json'

if(os.path.isfile(GeoJSON_File) == False):                                  #Check For File Existance
    print('Beginning file download with urllib2...')                        #Inform Us About Downloading Process
    urllib.request.urlretrieve(url, GeoJSON_File)                           #Saving File To Directory
    print('Download Completed!')                                            #Inform Completion

FileNotFoundError: [Errno 2] No such file or directory: 'Data/SpatialData/'

#### GIS Data (Third-level Administrative Divisions, Canada Toranto City, 2015)

##### Toranto Raw Map

In [ ]:
Toranto_map = folium.Map(location=[43.652051, -79.382646], zoom_start=11)
Toranto_map

In [11]:
Toranto_map = folium.Map(location=[43.652051, -79.382646], zoom_start=11)
folium.Choropleth(geo_data='Toranto.geojson',
                  key_on='feature.properties.DISTRICT'
                   ).add_to(Toranto_map)
Toranto_map

### 2.3.2. Loading Borough, Latitude and Longitude Data

On the following wikipedia page:  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  
There is a list of postal codes of Canada which will help us in preperation the data.  

In [22]:
Table_Class = "wikitable sortable"

wikipedia_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  #Page Url
page = requests.get(wikipedia_url)                                               #Get the Page
doc = lh.fromstring(page.content)                                                #getting the contents of the website

#Now we will get the wikipedia table that has a class attribute thet contints "wikitable sortable"
results  = doc.xpath("//table[@class='%s']" % Table_Class )

table = results[0]          #Getting The Table Which Should be the First in List (as HTML Element)
raw = etree.tostring(table) #Getting Raw HTML From HTML Element
df = pd.read_html(raw)[0]   #Use Pandas 'read_html' Function to Convert From HTML to Dataframe

df.head()                   #Printing Result Dataframe

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 2.3.3. Cleaning Data
First we will delete every row that has 'Borough' value is not assign, then we will copy the value of 'Borough' column to 'Neighbourhood' that has no value and after that we will reset the dataframe index. Next is kind of tricky; we will group data based on 'Postcode' and aggregate the child value into one cell but separate them by a comma

In [23]:
df.drop(df.loc[df['Borough'] == "Not assigned"].index, inplace=True)         #Removing Rows That Has "Not assigned" Value
df.loc[df['Neighbourhood'] == "Not assigned" , 'Neighbourhood'] = df['Borough'];
df = df.reset_index(drop=True)                                               #Reseting Index
tmp = df.groupby(['Postcode'])['Neighbourhood'].agg(', '.join).reset_index() #Grouping Data
tmp['Borough'] = df.groupby(['Postcode'])['Borough'].first().reset_index()['Borough']
df = tmp                                 
df.shape

(103, 3)

### 2.3.4. Preparing Latitude and Longitude

In [24]:
lldf = pd.read_csv('Geospatial_Coordinates.csv')
df['Latitude'] = lldf.loc[lldf['Postal Code'] == df['Postcode'] , 'Latitude']
df['Longitude'] = lldf.loc[lldf['Postal Code'] == df['Postcode'] , 'Longitude']

df.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476


In [25]:
# create map of Toronto using latitude and longitude values
Toranto_map = folium.Map(location=[43.652051, -79.382646], zoom_start=11)

# add markers to map
for _latitude, _longitude, _borough, _neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(_neighborhood, _borough)
    label = folium.Popup(label, parse_html=True)
    
                                #In Order To Create a Circle Marker Like the One on Google Then We'll Use the Folium Object 'CircleMarker'
    folium.CircleMarker(        #And Add The Following Parameters: -
        [_latitude, _longitude],                #1- Latitude And Longitude
        radius=5,                               #2- The Radius of the Circle
        popup=label,                            #3- The Lable That Will Pop Up
        color='blue',                           #4- The Color of the Circle
        fill=True,                              #5- Set Fill to True In Order to Fill the Circle
        fill_color='#3186cc',                   #6- This Parameter Represents the Filling Color
        fill_opacity=0.7,                       #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(Toranto_map)                   #Lastly We Assign the Marker to Our Map
    
Toranto_map

### 2.4. Getting Data About Gyms In Every Nighborhood

Getting All Toranto Gyms

In [16]:
CLIENT_ID = 'JYOC5WGRXKGGJN1LVBVVHA5RHQHCGSENQK2BCKKVNUQMHA2N'         # your Foursquare ID
CLIENT_SECRET = '0PI04RLZDG1SLY2DB2UV0ON45VHOH2QPDSDC00HPHESMRDYQ'     # your Foursquare Secret

In [17]:
# create URL ////43.562775, -79.589391 ///43.875118, -79.127508
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v=20180605&intent=match&ll=43.652051, -79.382646&city={}&query=gym&Limit=500&radius=10000'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    'Toronto')
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JYOC5WGRXKGGJN1LVBVVHA5RHQHCGSENQK2BCKKVNUQMHA2N&client_secret=0PI04RLZDG1SLY2DB2UV0ON45VHOH2QPDSDC00HPHESMRDYQ&v=20180605&intent=match&ll=43.652051, -79.382646&city=Toronto&query=gym&Limit=500&radius=10000'

In [29]:
#Scarborough, North York, East York, East Toronto,Central Toronto, Downtown Toronto, York, West Toronto,Queen's Park, Mississauga, Etobicoke
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7818cadb04f505a3602496'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'query': 'gym',
  'totalResults': 279,
  'suggestedBounds': {'ne': {'lat': 43.74205109000009,
    'lng': -79.25849056966459},
   'sw': {'lat': 43.56205090999991, 'lng': -79.5068014303354}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dc842f2e4cd5bc076711d90',
       'name': 'GoodLife Fitness',
       'location': {'address': '137 Yonge St',
        'crossStreet': 'at Richmond St',
        'lat': 43.65113563166627,
        'lng': -79.37879687835455,
        'labeledLatLngs': [{'label

In [187]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()
nearby_venues.shape

(30, 21)

<strong>#Returned (30, 23)</strong>

In [188]:
filtered_columns = ['venue.name' , 'venue.location.neighborhood' , 'venue.categories', 
                    'venue.location.address', 'venue.location.crossStreet',
                    'venue.location.formattedAddress'
                    ,'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,neighborhood,categories,address,crossStreet,formattedAddress,postalCode,lat,lng
0,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",137 Yonge St,at Richmond St,"[137 Yonge St (at Richmond St), Toronto ON M5C...",M5C 1W6,43.651136,-79.378797
1,Equinox Bay Street,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",199 Bay St,"at Commerce Court West, PATH Level","[199 Bay St (at Commerce Court West, PATH Leve...",M5L 1L5,43.648100,-79.379989
2,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",555 Richmond St W,Bathurst and Richmond,"[555 Richmond St W (Bathurst and Richmond), To...",M5V 3B1,43.646556,-79.402660
3,Adelaide Club Toronto,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",1 First Canadian Place,NaN,"[1 First Canadian Place, Toronto ON M5X 1C8, C...",M5X 1C8,43.649279,-79.381921
4,Totum Life Science King,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",445 King St. W,at Spadina Ave.,"[445 King St. W (at Spadina Ave.), Toronto ON ...",M5V 1K4,43.645151,-79.395918


### 2.4.1. Cleaning Venues By Category  

In here we'll need to ensure that every venue in the dataframe belongs to one of the following category's:

So the first thing that we'll need to do is crating two function for extracting category

In [35]:
#Used for getting category name
def get_category_name(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
#Used for getting category Id
def get_category_id(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['id']

In [189]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_id, axis=1)
nearby_venues['venue.categorie.name'] = nearby_venues.apply(get_category_name, axis=1)
nearby_venues.head()

,name,neighborhood,categories,address,crossStreet,formattedAddress,postalCode,lat,lng,venue.categories,venue.categorie.name
0,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",137 Yonge St,at Richmond St,"[137 Yonge St (at Richmond St), Toronto ON M5C...",M5C 1W6,43.651136,-79.378797,4bf58dd8d48988d176941735,Gym
1,Equinox Bay Street,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",199 Bay St,"at Commerce Court West, PATH Level","[199 Bay St (at Commerce Court West, PATH Leve...",M5L 1L5,43.648100,-79.379989,4bf58dd8d48988d176941735,Gym
2,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",555 Richmond St W,Bathurst and Richmond,"[555 Richmond St W (Bathurst and Richmond), To...",M5V 3B1,43.646556,-79.402660,4bf58dd8d48988d176941735,Gym
3,Adelaide Club Toronto,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",1 First Canadian Place,NaN,"[1 First Canadian Place, Toronto ON M5X 1C8, C...",M5X 1C8,43.649279,-79.381921,4bf58dd8d48988d175941735,Gym / Fitness Center
4,Totum Life Science King,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",445 King St. W,at Spadina Ave.,"[445 King St. W (at Spadina Ave.), Toronto ON ...",M5V 1K4,43.645151,-79.395918,4bf58dd8d48988d176941735,Gym


### 2.4.2. Formatting Venues PostalCode

In [190]:
Gym_per_Post  = nearby_venues
Gym_per_Post.postalCode = Gym_per_Post.postalCode.str.slice(stop=3).str.upper()
Gym_per_Post.head(100)

,name,neighborhood,categories,address,crossStreet,formattedAddress,postalCode,lat,lng,venue.categories,venue.categorie.name
0,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",137 Yonge St,at Richmond St,"[137 Yonge St (at Richmond St), Toronto ON M5C...",M5C,43.651136,-79.378797,4bf58dd8d48988d176941735,Gym
1,Equinox Bay Street,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",199 Bay St,"at Commerce Court West, PATH Level","[199 Bay St (at Commerce Court West, PATH Leve...",M5L,43.648100,-79.379989,4bf58dd8d48988d176941735,Gym
2,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",555 Richmond St W,Bathurst and Richmond,"[555 Richmond St W (Bathurst and Richmond), To...",M5V,43.646556,-79.402660,4bf58dd8d48988d176941735,Gym
3,Adelaide Club Toronto,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",1 First Canadian Place,NaN,"[1 First Canadian Place, Toronto ON M5X 1C8, C...",M5X,43.649279,-79.381921,4bf58dd8d48988d175941735,Gym / Fitness Center
4,Totum Life Science King,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",445 King St. W,at Spadina Ave.,"[445 King St. W (at Spadina Ave.), Toronto ON ...",M5V,43.645151,-79.395918,4bf58dd8d48988d176941735,Gym
5,barre3 Toronto - King St. West,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",325 King Street West,NaN,"[325 King Street West, Toronto ON M5V 1M3, Can...",M5V,43.646318,-79.390940,4bf58dd8d48988d175941735,Gym / Fitness Center
6,Fuel Training Club,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",45 camden st unit 100,spadina,"[45 camden st unit 100 (spadina), Toronto ON m...",M5V,43.646823,-79.397917,4bf58dd8d48988d176941735,Gym
7,Superclub,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",35 Mariner Terrace,Spadina Ave and Bremner Blvd,[35 Mariner Terrace (Spadina Ave and Bremner B...,NaN,43.639761,-79.391143,4bf58dd8d48988d176941735,Gym
8,Equinox Yorkville,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",55 Avenue Road,NaN,"[55 Avenue Road, Toronto ON M5R 1B9, Canada]",M5R,43.671220,-79.394963,4bf58dd8d48988d176941735,Gym
9,Moksha Yoga Downtown,NaN,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",577 Wellington St W,at Bathurst St,"[577 Wellington St W (at Bathurst St), Toronto...",M5V,43.642353,-79.403185,4bf58dd8d48988d102941735,Yoga Studio


### 2.4.3. Getting Gyms Count Per Venue

In [182]:
Gym_per_Post.postalCode.value_counts(dropna=False).to_frame()

,postalCode
M5V,9
M5C,2
M6J,2
M4Y,2
M6K,2
M6H,1
M5R,1
M5L,1
M4W,1
M5S,1


### 2.4.4. Filling Nan Values

In [167]:
def Calculate_Distant(lat2,lon2):
    # approximate radius of earth in km
    R = 6373.0
    _MinDis = 10000000000000000000000000
    _MinPC = ""
    for lat1,lon1,_pc in zip(df['Latitude'],df['Longitude'],df['Postcode']):
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = R * c
        if(_MinDis >= distance):
            _MinDis = distance
            _MinPC = _pc
    return _MinPC

In [191]:
_nan = Gym_per_Post.loc[Gym_per_Post.postalCode.isnull(), ['name','formattedAddress','lat','lng']]
for lat,lng,index in zip(_nan.lat,_nan.lng,_nan.index):
    val = Calculate_Distant(lat,lng)
    Gym_per_Post.postalCode[index] = val
    print(val)

,name,formattedAddress,lat,lng
7,Superclub,[35 Mariner Terrace (Spadina Ave and Bremner B...,43.639761,-79.391143
16,Cardio-Go,"[266 King Street West (Duncan), Toronto ON, Ca...",43.647017,-79.388143
18,Reebok Crossfit Liberty Village,"[Liberty Village, Toronto ON, Canada]",43.637036,-79.424802
26,Toronto Lawn Tennis Club,"[44 Price St. (at Yonge St.), Toronto ON, Canada]",43.680667,-79.388559


In [173]:
Gym_per_Post.head(100)

,name,neighborhood,categories,address,crossStreet,formattedAddress,postalCode,lat,lng,venue.categories,venue.categorie.name
0,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",137 Yonge St,at Richmond St,"[137 Yonge St (at Richmond St), Toronto ON M5C...",M5C,43.651136,-79.378797,4bf58dd8d48988d176941735,Gym
1,Equinox Bay Street,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",199 Bay St,"at Commerce Court West, PATH Level","[199 Bay St (at Commerce Court West, PATH Leve...",M5L,43.648100,-79.379989,4bf58dd8d48988d176941735,Gym
2,GoodLife Fitness,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",555 Richmond St W,Bathurst and Richmond,"[555 Richmond St W (Bathurst and Richmond), To...",M5V,43.646556,-79.402660,4bf58dd8d48988d176941735,Gym
3,Adelaide Club Toronto,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",1 First Canadian Place,NaN,"[1 First Canadian Place, Toronto ON M5X 1C8, C...",M5X,43.649279,-79.381921,4bf58dd8d48988d175941735,Gym / Fitness Center
4,Totum Life Science King,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",445 King St. W,at Spadina Ave.,"[445 King St. W (at Spadina Ave.), Toronto ON ...",M5V,43.645151,-79.395918,4bf58dd8d48988d176941735,Gym
5,barre3 Toronto - King St. West,NaN,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",325 King Street West,NaN,"[325 King Street West, Toronto ON M5V 1M3, Can...",M5V,43.646318,-79.390940,4bf58dd8d48988d175941735,Gym / Fitness Center
6,Fuel Training Club,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",45 camden st unit 100,spadina,"[45 camden st unit 100 (spadina), Toronto ON m...",M5V,43.646823,-79.397917,4bf58dd8d48988d176941735,Gym
7,Superclub,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",35 Mariner Terrace,Spadina Ave and Bremner Blvd,[35 Mariner Terrace (Spadina Ave and Bremner B...,M5J,43.639761,-79.391143,4bf58dd8d48988d176941735,Gym
8,Equinox Yorkville,NaN,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",55 Avenue Road,NaN,"[55 Avenue Road, Toronto ON M5R 1B9, Canada]",M5R,43.671220,-79.394963,4bf58dd8d48988d176941735,Gym
9,Moksha Yoga Downtown,NaN,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",577 Wellington St W,at Bathurst St,"[577 Wellington St W (at Bathurst St), Toronto...",M5V,43.642353,-79.403185,4bf58dd8d48988d102941735,Yoga Studio


### 2.5. Preparing the Data For The Map

In [174]:
GC = Gym_per_Post.postalCode.value_counts(dropna=False)
GC = pd.DataFrame({'Postcode':GC.index, 'Count':GC.values})
Final = pd.merge(df,GC , on ='Postcode')
Final.head(100)

,Postcode,Neighbourhood,Borough,Latitude,Longitude,dlat,dlon,Count
0,M4M,Studio District,East Toronto,43.659526,-79.340923,0.019764,0.050220,1
1,M4T,"Moore Park, Summerhill East",Central Toronto,43.689574,-79.383160,0.049813,0.007983,1
2,M4W,Rosedale,Downtown Toronto,43.679563,-79.377529,0.039801,0.013614,1
3,M4Y,Church and Wellesley,Downtown Toronto,43.665860,-79.383160,0.026099,0.007983,2
4,M5A,"Harbourfront, Regent Park",Downtown Toronto,43.654260,-79.360636,0.014499,0.030507,1
5,M5C,St. James Town,Downtown Toronto,43.651494,-79.375418,0.011733,0.015725,2
6,M5G,Central Bay Street,Downtown Toronto,43.657952,-79.387383,0.018191,0.003761,1
7,M5H,"Adelaide, King, Richmond",Downtown Toronto,43.650571,-79.384568,0.010810,0.006576,1
8,M5J,"Harbourfront East, Toronto Islands, Union Station",Downtown Toronto,43.640816,-79.381752,0.001054,0.009391,1
9,M5K,"Design Exchange, Toronto Dominion Centre",Downtown Toronto,43.647177,-79.381576,0.007415,0.009567,1


### 2.6. Visualize the Map

In [214]:
Toranto_map = folium.Map(location=[43.652051, -79.382646], zoom_start=11) #Creating New Folium Map Object

folium.Choropleth(geo_data='Toranto.geojson',
                  key_on='feature.properties.DISTRICT',
                  fill_opacity=0.3,
                  line_opacity=0.8).add_to(Toranto_map)

lat = []                                                                  #A List for Calculating MidPoint
long = []                                                                 #A List For Calculating MidPoint



for _latitude, _longitude  in zip(Final['Latitude'], Final['Longitude']):
    lat.append(_latitude)                        #Appand Latitude
    long.append(_longitude)                      #Appand Logitude 

Range_Latitude = sum(lat)/len(lat)              #Calculating MidPoint Latitude
Range_Longitude = sum(long)/len(long)           #Calculating MidPoint Latitude

#Now to Calclate the Radius We'll Need to Identify the Fearthist Point From MidPoint
MidPoint_Radius = 100


#Low Level 
folium.CircleMarker(        
        [Range_Latitude, Range_Longitude],      #1- Latitude And Longitude
        radius=MidPoint_Radius * 3.5,                 #2- The Radius of the Circle
        popup="Low level",                       #3- The Lable That Will Pop Up
        color='Yellow',                         #4- The Color of the Circle
        fill=False,                             #5- Set Fill to True In Order to Fill the Circle
        fill_color='Yellow',                    #6- This Parameter Represents the Filling Color
        fill_opacity=0.1666,                       #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(Toranto_map)                   #Lastly We Assign the Marker to Our Map

#Mid Level 
folium.CircleMarker(        
        [Range_Latitude, Range_Longitude],      #1- Latitude And Longitude
        radius=MidPoint_Radius * 2.25,                 #2- The Radius of the Circle
        popup="Mid level",                       #3- The Lable That Will Pop Up
        color='Orange',                         #4- The Color of the Circle
        fill=False,                             #5- Set Fill to True In Order to Fill the Circle
        fill_color='Orange',                    #6- This Parameter Represents the Filling Color
        fill_opacity=0.111,                       #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(Toranto_map)                   #Lastly We Assign the Marker to Our Map

#High Level 
folium.CircleMarker(        
        [Range_Latitude, Range_Longitude],      #1- Latitude And Longitude
        radius=MidPoint_Radius,                 #2- The Radius of the Circle
        popup="High level",                       #3- The Lable That Will Pop Up
        color='IndianRed',                         #4- The Color of the Circle
        fill=False,                             #5- Set Fill to True In Order to Fill the Circle
        fill_color='IndianRed',                    #6- This Parameter Represents the Filling Color
        fill_opacity=0.0555555,                       #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(Toranto_map)                   #Lastly We Assign the Marker to Our Map


# add markers to map
for _latitude, _longitude, _borough, _neighborhood , _pc in zip(Final['Latitude'], Final['Longitude'], Final['Borough'], Final['Neighbourhood'], Final['Postcode']):
    label = '{}, {}'.format(_neighborhood, _borough)
    label = folium.Popup(label, parse_html=True)

                                #In Order To Create a Circle Marker Like the One on Google Then We'll Use the Folium Object 'CircleMarker'
    folium.CircleMarker(        #And Add The Following Parameters: -
        [_latitude, _longitude],                #1- Latitude And Longitude
        radius=5,                               #2- The Radius of the Circle
        popup=label,                            #3- The Lable That Will Pop Up
        color='Red',                            #4- The Color of the Circle
        fill=True,                              #5- Set Fill to True In Order to Fill the Circle
        fill_color='Gray',                       #6- This Parameter Represents the Filling Color
        fill_opacity=1,                         #7- The Level of Filling Opacity (Value Between 0 and 1)
        parse_html=False,                       #8-
        ).add_to(Toranto_map)                   #Lastly We Assign the Marker to Our Map


Toranto_map